In [1]:
import pymysql
from tabulate import tabulate
import matplotlib.pyplot as plt

connection = pymysql.connect(
    user='root',
    password='root',
    host='localhost',
    database='library_app'
)
cursor = connection.cursor()

def Issue_book():
    print("Enter book issue details:")
    try:
        member_id = int(input("Enter member ID: "))
        book_id = int(input("Enter book ID: "))
        
        # Check if the member exists
        select_member = "SELECT * FROM member WHERE member_id = %s"
        cursor.execute(select_member, (member_id,))
        member_record = cursor.fetchone()
        
        # Check if the book exists and has available copies
        select_book = "SELECT * FROM book WHERE book_id = %s AND Rem_copies > 0"
        cursor.execute(select_book, (book_id,))
        book_record = cursor.fetchone()
        
        if member_record and book_record:
            issue_id = int(input("Enter issue ID: "))
            issue_date = input("Enter a date in YYYY-MM-DD format: ")
            copies = int(input("Enter number of copies for issue: "))
            
            if copies > 0 and copies <= book_record[5]:  # Assuming Rem_copies is at index 5
                # Insert into issue table
                insert_issue = 'INSERT INTO issue (issue_id, issue_date, member_id, book_id, copies) VALUES (%s, %s, %s, %s, %s)'
                insert_values_issue = (issue_id, issue_date, member_id, book_id, copies)
                cursor.execute(insert_issue, insert_values_issue)
                
                # Update Rem_copies in the book table
                update_copies = "UPDATE book SET Rem_copies = Rem_copies - %s WHERE book_id = %s"
                cursor.execute(update_copies, (copies, book_id))
                
                connection.commit()
                print("Book issuance successful!")
            else:
                print("Invalid number of copies or insufficient.")
        else:
            print('No member exists or not available ')

    except pymysql.connect.Error as error:
        print(f"MySQL error: {error}")

def return_book():
    print("Enter book return details:")
    try:
        issue_id = int(input("Enter  issue ID: "))
        
        select_issue = "SELECT * FROM issue WHERE issue_id = %s"  
        cursor.execute(select_issue, (issue_id,))
        issue_record = cursor.fetchone()
        
        if issue_record:
            print("Issue Record:", issue_record)
            
            # Accessing specific indices
            copies = issue_record[4]  # Assuming copies are at index 4
            book_id = issue_record[3]  # Assuming book_id is at index 3

            print("Number Copies Returned:", copies)
            print("Book ID:", book_id)

            # Prompt the user for return_id
            return_id = int(input("Enter Return ID: "))

            # Insert into re_turn table
            return_date = input("Enter a date in YYYY-MM-DD format: ")
            member_id = int(input("Enter member ID: "))
            book_id = int(input("Enter book ID: "))
            insert_return = 'INSERT INTO re_turn (return_id, return_date, member_id, book_id, copies) VALUES (%s, %s, %s, %s, %s)'
            insert_values_return = (return_id, return_date, member_id, book_id, copies)
            cursor.execute(insert_return, insert_values_return)

            # Delete from issue table
            delete_issue = "DELETE FROM issue WHERE issue_id = %s"
            cursor.execute(delete_issue, (issue_id,))

            # Update Rem_copies in the book table
            update_copies = "UPDATE book SET Rem_copies = Rem_copies + %s WHERE book_id = %s"
            cursor.execute(update_copies, (copies, book_id))

            connection.commit()
            print("Book returned successful!")
        else:
            print('No other issued book exists')

    except pymysql.connect.Error as error:
        print(f"MySQL error: {error}")

def issue_status_by_date():
    print("Enter date to check issue status:")
    try:
        issue_date = input("Enter a date in YYYY-MM-DD format: ")
        query = "SELECT i.book_id, b.Bname, m.Mname, i.copies FROM issue i JOIN book b ON i.book_id = b.book_id JOIN member m ON i.member_id = m.member_id WHERE i.issue_date = %s"
        cursor.execute(query, (issue_date,))
        results = cursor.fetchall()
        headers = ["Book ID", "Book Name", "Member Name", "Copies Issued"]
        print(tabulate(results, headers=headers, tablefmt="pretty"))
    except pymysql.connect.Error as error:
        print(f"MySQL error: {error}")

def return_details_by_date():
    print("Enter date to check return details:")
    try:
        return_date = input("Enter a date in YYYY-MM-DD format: ")
        query = "SELECT r.book_id, b.Bname, m.Mname, r.copies FROM re_turn r JOIN book b ON r.book_id = b.book_id JOIN member m ON r.member_id = m.member_id WHERE r.return_date = %s"
        cursor.execute(query, (return_date,))
        results = cursor.fetchall()
        headers = ["Book ID", "Book Name", "Member Name", "Copies Returned"]
        print(tabulate(results, headers=headers, tablefmt="pretty"))
    except pymysql.connect.Error as error:
        print(f"MySQL error: {error}")

print("Done.")

Done.
